### Importing the libraries

In [17]:
import pandas as pd
import numpy as np
import google.generativeai as genai
from dotenv import dotenv_values

pd.set_option('display.max_colwidth', None)

### Importing the movie dataset

In [18]:
df = pd.read_csv('dataset/tmdb_5000_movies.csv')

In [19]:
columns = ['title', 'overview']

In [20]:
df = df[columns]

In [21]:
df.head()

,title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems."
2,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE."
3,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy."
4,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands."


In [22]:
df.shape

(4803, 2)

In [23]:
df.isna().sum()

title       0
overview    3
dtype: int64

In [24]:
df.dropna(inplace=True)

In [25]:
df = df.sample(100)

In [26]:
df.sample(10)

,title,overview
3110,Welcome to the Rileys,"Years after their teenage daughter’s death, Lois and Doug Riley, an upstanding Indiana couple, are frozen by estranging grief. Doug escapes to New Orleans on a business trip. Compelled by urgencies he doesn’t understand, he insinuates himself into the life of an underage hooker, becoming her platonic guardian."
3560,Ouija,A group of friends must confront their most terrifying fears when they awaken the dark powers of an ancient spirit board.
1520,The Incredible Burt Wonderstone,"After breaking up with his longtime stage partner, a famous but jaded Vegas magician fights for relevance when a new, ""hip"" street magician appears on the scene."
3874,The Opposite Sex,"A Bet's A Bet tells the story of Vince, New England's most successful divorce attorney. To Vince, life is one big competition, and losing is unacceptable. This also applies in his dating life with his love 'em and leave 'em approach. Then Vince meets Jane, who is beautiful, successful and also extremely driven. Together they enter into a series of entertaining wagers with each other where the winner gets to decide the fate of the loser. After all, ""A Bet's a Bet!"""
2995,Mad Max Beyond Thunderdome,"Mad Max becomes a pawn in a decadent oasis of a technological society, and when exiled, becomes the deliverer of a colony of children."
3556,10 Cloverfield Lane,"After a car accident, Michelle awakens to find herself in a mysterious bunker with two men named Howard and Emmett. Howard offers her a pair of crutches to help her remain mobile with her leg injury sustained from the car crash and tells her to ""get good on those"" before leaving the bunker. She has been given the information that there has been an alien attack and the outside world is poisoned. However, Howard and Emmett's intentions soon become questionable and Michelle is faced with a question: Is it better in here or out there?"
894,Bulletproof Monk,"A mysterious and immortal Tibetan kung fu master, who has spent the last 60 years traveling around the world protecting the ancient Scroll of the Ultimate, mentors a selfish street kid in the ancient intricacies of kung fu."
65,The Dark Knight,"Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."
105,Alice Through the Looking Glass,"In the sequel to Tim Burton's ""Alice in Wonderland"", Alice Kingsleigh returns to Underland and faces a new adventure in saving the Mad Hatter."
1614,Take the Lead,"A former professional dancer volunteers to teach dance in the New York public school system and, while his background first clashes with his students' tastes, together they create a completely new style of dance. Based on the story of ballroom dancer, Pierre Dulane."


### Embedding the dataframe

In [27]:
ENV = dotenv_values(".env")
api_key = ENV['API_KEY']
genai.configure(api_key=api_key)

In [28]:
def embed_content(title, text, model, task_type):
    if text is None:
        return genai.embed_content(
            model=model,
            content=text,
            task_type=task_type,
        )["embedding"]
    else:
        return genai.embed_content(
            model=model,
            title=title,
            content=text,
            task_type=task_type,
        )["embedding"]

In [29]:
embedding_model = "models/embedding-001"
df["Embeddings"] = df.apply(
    lambda x: embed_content(
        x["title"],
        x["overview"],
        embedding_model,
        "RETRIEVAL_DOCUMENT",
    ),
    axis=1,
)

In [30]:
def get_suggestion(query, df, model):
    query_embedding = embed_content(
        None,
        query,
        model,
        "RETRIEVAL_QUERY",
    )

    scalars = np.dot(np.stack(df["Embeddings"]), query_embedding)
    index = np.argmax(scalars)
    return df.iloc[index][["title", "overview"]]

In [41]:
query = "I like movies about rapper"
suggestion = get_suggestion(query, df, embedding_model)
print(f"Title: {suggestion.title}")
print(f"Description: {suggestion.overview}")

Title: 8 Mile
Description: The setting is Detroit in 1995. The city is divided by 8 Mile, a road that splits the town in half along racial lines. A young white rapper, Jimmy "B-Rabbit" Smith Jr. summons strength within himself to cross over these arbitrary boundaries to fulfill his dream of success in hip hop. With his pal Future and the three one third in place, all he has to do is not choke.


In [37]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1,
}

In [49]:
safety_settings = {
    'HARASSMENT': 'BLOCK_MEDIUM_AND_ABOVE',
    'HATE': 'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL': 'BLOCK_MEDIUM_AND_ABOVE',
    'DANGEROUS': 'BLOCK_MEDIUM_AND_ABOVE',
}

In [43]:
prompt = f"Com base na [QUERY] sobre características/atributos de filmes: {query}. Elabore um breve resumo do filme: {suggestion.title}, considerando as características citadas na [QUERY]. Seja simpático e divertido, e busco me convencer a assistir a esse filme"

In [50]:
model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [51]:
response = model.generate_content(prompt)

In [52]:
print(response.text)

Prepare-se para uma jornada eletrizante em 8 Mile, onde o mundo do rap e a luta pessoal colidem!

Siga a história de Jimmy "B-Rabbit" Smith Jr., um jovem branco que tenta conquistar o respeito no underground do rap de Detroit. Apesar dos obstáculos e do preconceito, B-Rabbit tem um dom inegável para as palavras e uma determinação inabalável.

Testemunhe batalhas de rap épicas, onde B-Rabbit enfrenta rivais implacáveis e prova sua habilidade lírica. Sinta a emoção enquanto ele luta contra seus demônios internos e tenta encontrar seu lugar em um mundo que não parece recebê-lo.

Com um elenco estelar liderado por Eminem, 8 Mile é um filme inspirador e comovente que irá cativá-lo do início ao fim. Então, prepare-se para rimar, batalhar e torcer por B-Rabbit enquanto ele busca seu sonho de se tornar um rapper respeitado em 8 Mile!
